## **Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ**



Основным заданием будет обучить классификатор на основе сверточных сетей, чтобы научиться отличать всех жителей Спрингфилда. мин пороговый скор 0,95. 

сабмит грузится на каггл
https://www.kaggle.com/c/journey-springfield/overview

In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

import numpy as np
import matplotlib.pyplot as plt
import time
import os
import pickle
from skimage import io
from tqdm import tqdm, tqdm_notebook
from PIL import Image
from pathlib import Path
from multiprocessing.pool import ThreadPool
from sklearn.preprocessing import LabelEncoder
from matplotlib import colors, pyplot as plt
%matplotlib inline

# в sklearn не все гладко, чтобы в colab удобно выводить картинки 
# мы будем игнорировать warnings
import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning)

plt.ion()

In [2]:
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [3]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [4]:
!unzip -q '/content/gdrive/My Drive/dl_stepik/homework_11_simpsons/journey-springfield.zip' 

In [3]:
!ls train

simpsons_dataset


In [4]:
!nvidia-smi
#import torch
torch.cuda.is_available()

Fri Aug 14 11:28:08 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P8    11W /  70W |     10MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

True

В тесте будет 990 картнок, для которых вам будет необходимо предсказать класс.

In [5]:
# разные режимы датасета 
DATA_MODES = ['train', 'val', 'test']
# все изображения будут масштабированы к размеру 224x224 px
#RESCALE_SIZE = 224
# работаем на видеокарте
DEVICE = torch.device("cuda")

https://jhui.github.io/2018/02/09/PyTorch-Data-loading-preprocess_torchvision/

Ниже мы исспользуем враппер над датасетом для удобной работы. Вам стоит понимать, что происходит с LabelEncoder и с torch.Transformation.

ToTensor конвертирует PIL Image с параметрами в диапазоне [0, 255] (как все пиксели) в FloatTensor размера (C x H x W) [0,1] , затем производится масштабирование: $input = \frac{input - \mu}{\text{standard deviation}} \$, константы - средние и дисперсии по каналам на основе ImageNet

Стоит также отметить, что мы переопределяем метод getitem для удобства работы с данной структурой данных. Также используется LabelEncoder для преобразования строковых меток классов в id и обратно. В описании датасета указано, что картинки разного размера, так как брались напрямую с видео, поэтому следуем привести их к одному размер (это делает метод _prepare_sample)

In [6]:
class SimpsonsDataset_new(Dataset):
    """
    Датасет с картинками, который паралельно подгружает их из папок
    производит скалирование и превращение в торчевые тензоры
    """
    def __init__(self, files, mode):
        super().__init__()
        # список файлов для загрузки
        self.files = sorted(files)
        # режим работы
        self.mode = mode

        if self.mode not in DATA_MODES:
            print(f"{self.mode} is not correct; correct modes: {DATA_MODES}")
            raise NameError

        self.len_ = len(self.files) 
        self.label_encoder = LabelEncoder()

        if self.mode != 'test':
            self.labels = [path.parent.name for path in self.files]
            self.label_encoder.fit(self.labels)
            # Кодируем наши классы числами, и сохраняем в файл
            # print(dict(zip(self.label_encoder.classes_, self.label_encoder.transform(self.label_encoder.classes_))))
            with open('label_encoder.pkl', 'wb') as le_dump_file:
                  pickle.dump(self.label_encoder, le_dump_file)
    # переопределяем метод len как количество файлов                  
    def __len__(self):
        return self.len_
    # подгружаем и возвращает картинку  
    def load_sample(self, file):
        image = Image.open(file)
        image.load()
        return image
  
    def __getitem__(self, index):
        # для преобразования изображений в тензоры PyTorch и нормализации входа
        if self.mode == 'train':
            transform = transforms.Compose([
              #transforms.RandomRotation(25),
              transforms.RandomHorizontalFlip(),
              transforms.Resize((299,299)),
              transforms.ToTensor(),
              transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
        ])
        if self.mode == 'vale' or 'test':
            transform = transforms.Compose([
              transforms.Resize((299,299)),
              transforms.ToTensor(),
              transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
        ])
        # подгружаем картинку
        x = self.load_sample(self.files[index])
        # ресайзим и возвращаем
        #x = self._prepare_sample(x) прям в композе засунем 
        # нормализуем
        #x = np.array(x / 255, dtype='float32') transforms.ToTensor() итак переводит из диапазона [0, 255] в диапазон [0.0, 1.0]
        # трансформируем по функии выше
        x = transform(x)
        if self.mode == 'test':
            return x
        else:
            label = self.labels[index]
            label_id = self.label_encoder.transform([label])
            y = label_id.item()
            return x, y
    # ресайз    
    def _prepare_sample(self, image):
        image = image.resize((RESCALE_SIZE, RESCALE_SIZE))
        return np.array(image)

OverSampling (Самописный велосипед =)

Он заключается в том, чтобы докопировать количество картинок из меньших классов,так, чтобы общее число картинок было таким же как и у классов с большим числом картинок.

Если мы поступим таким образом наш dataset разрастется до таких размеров, что перестанет влазить в оперативную память видеокарты.

Можно однако сделать по другому, глядя на полученные нами таблицы точности мы видим, что без балансировки все классы, где было не менее 100 картинок, отлично распознаются.

Т.е. мы просто откопируем картинки малочисленных классов менее 100 картинок из выборки train, так чтобы число на каждый малочисленный класс было не менее 100.

Если мы сделаем oversampling, - возьмем картинки из всей выборки до разбиения ее на 'train' и 'val' то скорее всего в train попадут и картинки из выборки val, а мы этого не хотим.

При этом у нас все так же будет работать аугментация, да картинки в малочисленных классах будут одинаковыми, но при каждой загрузке одни и те же картинки будут по разному аугментироваться, и будут отличаться друг от друга. Конечно такого разнообразия как у изначально многочисленных классов не будет, но это лучше чем ничего.

Напишем такую функцию вручную. Нам не надо копировать картинки, нам надо копировать пути к картинкам !

In [7]:
TRAIN_DIR = Path('train/simpsons_dataset')
TEST_DIR = Path('testset/testset')

train_val_files = sorted(list(TRAIN_DIR.rglob('*.jpg')))
test_files = sorted(list(TEST_DIR.rglob('*.jpg')))

In [8]:
from sklearn.model_selection import train_test_split
train_val_labels = [path.parent.name for path in train_val_files] # классы train + val
train_files, val_files = train_test_split(train_val_files, test_size=0.25, \
                                          stratify=train_val_labels)

In [9]:
train_labels = [path.parent.name for path in train_files] # классы train
val_labels = [path.parent.name for path in val_files]     # классы val

In [10]:
def create_dct_path_labels(train_files, train_labels):
    dct_simpsons = {}
    for label_i in np.unique(train_labels).tolist():
        dct_simpsons[label_i] = []

    for path_i, label_i in zip(train_files, train_labels):
        dct_simpsons[label_i].append(path_i)

    return dct_simpsons

In [11]:
# Создадим словарь в котором ключами будут персонажи Симпсонов, а значениями списки с путями к картинкам.
dct_path_train = create_dct_path_labels(train_files, train_labels)

In [12]:
#Получился у нас словарь в нем ключ это персонаж из Симпсонов, а значение это список содержащий пути к картинкам
#list = [PosixPath('dataset/train/simpsons_dataset/waylon_smithers/pic_0033.jpg'), PosixPath('dataset/train/simpsons_dataset/waylon_smithers/pic_0139.jpg').....]

In [13]:
# Дополним картинки классов у которых менее 100 картинок, до 100 картинок в классе
for person in dct_path_train:
    if len(dct_path_train[person]) < 100:
        dct_path_train[person] = dct_path_train[person] * (100 // len(dct_path_train[person]))
        dct_path_train[person].extend(dct_path_train[person][:100 - len(dct_path_train[person])])

In [14]:
for person in dct_path_train:
    print(f"{person}\t{len(dct_path_train[person])}")

abraham_grampa_simpson	685
agnes_skinner	100
apu_nahasapeemapetilon	467
barney_gumble	100
bart_simpson	1006
carl_carlson	100
charles_montgomery_burns	895
chief_wiggum	739
cletus_spuckler	100
comic_book_guy	352
disco_stu	100
edna_krabappel	343
fat_tony	100
gil	100
groundskeeper_willie	100
homer_simpson	1684
kent_brockman	373
krusty_the_clown	904
lenny_leonard	233
lionel_hutz	100
lisa_simpson	1015
maggie_simpson	100
marge_simpson	968
martin_prince	100
mayor_quimby	185
milhouse_van_houten	809
miss_hoover	100
moe_szyslak	1089
ned_flanders	1090
nelson_muntz	269
otto_mann	100
patty_bouvier	100
principal_skinner	895
professor_john_frink	100
rainier_wolfcastle	100
ralph_wiggum	100
selma_bouvier	100
sideshow_bob	658
sideshow_mel	100
snake_jailbird	100
troy_mcclure	100
waylon_smithers	136


In [15]:
new_train_files = []

for person in dct_path_train:
    new_train_files.extend(dct_path_train[person])

In [16]:
#Отлично, мы видим, что теперь все классы имеют минимум 100 картинок. У нас получилось.

val_dataset = SimpsonsDataset_new(val_files, mode='val')
new_train_dataset = SimpsonsDataset_new(new_train_files, mode='train')

## **FIT**

In [17]:
inception_model = models.inception_v3(pretrained=True,aux_logits=False)

Finally, notice that inception_v3 requires the input size to be (299,299), whereas all of the other models expect (224,224).

In [18]:
def fit_epoch(model, train_loader, criterion, optimizer):
    running_loss = 0.0
    running_corrects = 0
    processed_data = 0
  
    for inputs, labels in train_loader:
       # батчи, 3цвета, размер картинки torch.Size([128, 3, 299, 299]) / батча 128 должно хватить(не всю сеть переобучать буду)
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        optimizer.zero_grad()

        outputs = model(inputs) # инспетион/ 128 картинок с вероятностью для каждого класса torch.Size([128, 42])
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        preds = torch.argmax(outputs, 1) # по каждой картинке макс. вероятность - класс
        running_loss += loss.item() * inputs.size(0) # умножамем лосс (энтропию) на рамзер батча
        running_corrects += torch.sum(preds == labels.data)
        processed_data += inputs.size(0)
              
    train_loss = running_loss / processed_data # средний лос по картинке
    train_acc = running_corrects.cpu().numpy() / processed_data # перевести в cpu (в куде не считает )  - процент прав. ответов
    return train_loss, train_acc

In [19]:
def eval_epoch(model, val_loader, criterion):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    processed_size = 0

    for inputs, labels in val_loader:
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)

        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            preds = torch.argmax(outputs, 1)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        processed_size += inputs.size(0)
    val_loss = running_loss / processed_size
    val_acc = running_corrects.double() / processed_size # is equivalent to self.to(torch.float64)
    return val_loss, val_acc

In [25]:
def train(train_files, val_files, model, epochs, batch_size):
    train_loader = DataLoader(new_train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    history = []
    log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} \
    val_loss {v_loss:0.4f} train_acc {t_acc:0.4f} val_acc {v_acc:0.4f}"

    with tqdm(desc="epoch", total=epochs) as pbar_outer:
        opt = torch.optim.AdamW(param_for_train,lr = 0.0001)
        criterion = nn.CrossEntropyLoss()
        scheduler=torch.optim.lr_scheduler.StepLR(opt,7,gamma=0.1)

        for epoch in range(epochs):
            train_loss, train_acc = fit_epoch(model, train_loader, criterion, opt)
            print("loss", train_loss)
            
            val_loss, val_acc = eval_epoch(model, val_loader, criterion)
            history.append((train_loss, train_acc, val_loss, val_acc))
            scheduler.step()
            # обновляем прогресс бар
            pbar_outer.update(1)
            tqdm.write(log_template.format(ep=epoch+1, t_loss=train_loss,\
                                           v_loss=val_loss, t_acc=train_acc, v_acc=val_acc))
            
    return history

про файн тюнинг https://medium.com/@14prakash/almost-any-image-classification-problem-using-pytorch-i-am-in-love-with-pytorch-26c7aa979ec4

https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html

In [21]:
## Freezing the first few layers. Here I am freezing the first 7 layers 
ct = 0
for name, child in inception_model.named_children():
    ct += 1
    if ct < 7:
        for name2, params in child.named_parameters():
          params.requires_grad = False

param_for_train = [i for i in inception_model.parameters() if i.requires_grad == True]

num_features2 = 2048
inception_model.fc = nn.Linear(num_features2, 42)
inception_model = inception_model.to(DEVICE)

In [26]:
history = train(new_train_dataset, val_dataset, model=inception_model, epochs=10, batch_size=128)

epoch:   0%|          | 0/10 [00:00<?, ?it/s]

loss 1.5197161421457597


epoch:  10%|█         | 1/10 [05:32<49:55, 332.86s/it]


Epoch 001 train_loss: 1.5197     val_loss 0.3749 train_acc 0.6890 val_acc 0.9278
loss 0.2350190382295369


epoch:  20%|██        | 2/10 [10:58<44:05, 330.72s/it]

...


Epoch 008 train_loss: 0.0063     val_loss 0.1200 train_acc 0.9983 val_acc 0.9757
loss 0.0019571092127405876


epoch:  90%|█████████ | 9/10 [49:06<05:27, 327.46s/it]


Epoch 009 train_loss: 0.0020     val_loss 0.1233 train_acc 0.9994 val_acc 0.9763
loss 0.0014988329549918207


epoch: 100%|██████████| 10/10 [54:32<00:00, 327.25s/it]


Epoch 010 train_loss: 0.0015     val_loss 0.1262 train_acc 0.9995 val_acc 0.9765


In [27]:
def predict(model, test_loader):
    with torch.no_grad():
        logits = []
    
        for inputs in test_loader:
            inputs = inputs.to(DEVICE)
            model.eval() # исключает в оценке ненужные слои например Dropouts Layers (выпали чассть нейронов)
            outputs = model(inputs).cpu()
            logits.append(outputs)
            
    probs = nn.functional.softmax(torch.cat(logits), dim=-1).numpy() #нужен софтмакс, тк. ентропия в лоссе а слоя нет
    return probs

In [28]:
label_encoder = pickle.load(open("label_encoder.pkl", 'rb'))

In [29]:
test_dataset = SimpsonsDataset_new(test_files, mode="test")
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=128)
probs = predict(inception_model, test_loader)

preds = label_encoder.inverse_transform(np.argmax(probs, axis=1))
test_filenames = [path.name for path in test_dataset.files]

In [31]:
import pandas as pd
df = pd.DataFrame()
df['Id'] = test_filenames
df['Expected'] = preds
df.to_csv('gdrive/My Drive/simpsons_simple_cnn_baseline_new.csv', index=False)